In [4]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
v0 = 0.5172
r = 0
T = 1 / 12
K = 1000
xi = 0.5
# symbolic variables
s, v, t = symbols('s v t')
sigma = sqrt(v0)
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + sigma**2 / 2) * t) / (sigma * sqrt(t))
delta0 = 0.5 * (v - sigma**2) * s * density(N)(d_p) / (sigma * sqrt(t))

def bs(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d_m = d_p - sigma * np.sqrt(T)
  return S * N(d_p) - K * np.exp(-r * T) * N(d_m)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

delta = [delta0]
delta_func = []
for _ in range(1, 5):
  delta.append(L(delta[-1]))
for i in range(0, 5):
  delta_func.append(lambdify([s, v, t], delta[i]))

S = np.arange(950, 1051, 10)
Approx_4 = []
for s in S:
  p0_approx = bs(s, K, T, r, np.sqrt(v0)) + T * delta_func[0](s, v0, T)
  p1_approx = p0_approx + T**2 / 2 * delta_func[1](s, v0, T)
  p2_approx = p1_approx + T**3 / 6 * delta_func[2](s, v0, T)
  p3_approx = p2_approx + T**4 / 24 * delta_func[3](s, v0, T)
  p4_approx = p3_approx + T**5 / 120 * delta_func[4](s, v0, T)
  Approx_4.append(p4_approx)

Monte_carlo = np.array([57.1354, 61.5947, 66.2564, 71.1264, 76.1974, 81.4727, 86.9598, 92.6641, 98.5726, 104.6802, 110.9633])
JFE_results = np.array([57.6931, 62.2185, 66.9456, 71.8731, 76.9990, 82.3211, 87.8366, 93.5424, 99.4349, 105.5104, 111.7646])
JFE_results_diff = np.array([0.9761, 1.0128, 1.0402, 1.0498, 1.0521, 1.0413, 1.0083, 0.9478, 0.8748, 0.7931, 0.7221])
Approx_diff = (Approx_4 - Monte_carlo) / Monte_carlo * 100

df = pd.DataFrame({"S": S, "Monte Carlo": Monte_carlo, "Replicate price": np.round(Approx_4, 4), "Replicate %diff": np.round(Approx_diff, 4), "JFE price": JFE_results, "JFE %diff": JFE_results_diff})
print(df)
df.to_csv("Table 2 Panel A.csv")

E:\Anaconda\lib\site-packages\numpy\__init__.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """
E:\Anaconda\lib\site-packages\numpy\__init__.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """


       S  Monte Carlo  Replicate price  Replicate %diff  JFE price  JFE %diff
0    950      57.1354    -1.159020e+08    -2.028551e+08    57.6931     0.9761
1    960      61.5947              NaN              NaN    62.2185     1.0128
2    970      66.2564    -1.269945e+08    -1.916714e+08    66.9456     1.0402
3    980      71.1264     2.743766e+07     3.857582e+07    71.8731     1.0498
4    990      76.1974    -5.222487e+07    -6.853902e+07    76.9990     1.0521
5   1000      81.4727     2.911861e+07     3.574023e+07    82.3211     1.0413
6   1010      86.9598     1.309857e+08     1.506278e+08    87.8366     1.0083
7   1020      92.6641     1.850123e+08     1.996590e+08    93.5424     0.9478
8   1030      98.5726     5.143612e+07     5.218085e+07    99.4349     0.8748
9   1040     104.6802     2.604393e+07     2.487941e+07   105.5104     0.7931
10  1050     110.9633     3.022409e+07     2.723782e+07   111.7646     0.7221
